In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import matplotlib.pyplot as plt

In [3]:
# 모델 정의
def modeling(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(128, 3, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(32, 3, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    if num_classes == 2:
        outputs = layers.Dense(1, activation="sigmoid")(x)
    else:
        outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [4]:
ver = 'v10'
work_dir = "/content/drive/MyDrive/고양이 안구 질환 데이터/"

diseases = ["안검염", "비궤양성각막염", "결막염", "각막부골편", "각막궤양"]
dis = diseases[0]   # 안검염

model_dir = work_dir + 'model/' + ver + '/'
model_name = 'model' + dis + ver

image_size = (360, 360)
batch_size = 32
num_classes = 2

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = [keras.callbacks.ModelCheckpoint(model_dir + "save_at_{epoch}.h5", save_best_only=True)]

# 모델 폴더 생성
if not os.path.exists(model_dir):
  os.makedirs(model_dir)

In [ ]:
# 데이터 불러오기
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_ds = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/고양이 안구 질환 데이터/Training/' + dis,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary'
        )

validation_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

val_ds = validation_datagen.flow_from_directory(
        '/content/drive/MyDrive/고양이 안구 질환 데이터/Validation/' + dis,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary'
        )

Found 1910 images belonging to 2 classes.
Found 246 images belonging to 2 classes.


In [ ]:
epochs = 30
model = modeling(input_shape = image_size + (3,), num_classes=num_classes)

# 모델 학습
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"],)
result = model.fit(train_ds,epochs=epochs,callbacks=[early_stopping, model_checkpoint],validation_data=val_ds,)

# 모델 저장
model.save(model_dir + model_name)

Epoch 1/30
60/60 [==============================] - 1598s 26s/step - loss: 2.0895 - accuracy: 0.5089 - val_loss: 1.5597 - val_accuracy: 0.6301
Epoch 2/30
60/60 [==============================] - 1615s 27s/step - loss: 1.4572 - accuracy: 0.5178 - val_loss: 1.3828 - val_accuracy: 0.4878
Epoch 3/30
60/60 [==============================] - 1496s 25s/step - loss: 1.3184 - accuracy: 0.5246 - val_loss: 1.2745 - val_accuracy: 0.4797
Epoch 4/30
60/60 [==============================] - 1494s 25s/step - loss: 1.2281 - accuracy: 0.5723 - val_loss: 1.1989 - val_accuracy: 0.4431
Epoch 5/30
60/60 [==============================] - 1497s 25s/step - loss: 1.1552 - accuracy: 0.6272 - val_loss: 1.1467 - val_accuracy: 0.4634
Epoch 6/30
60/60 [==============================] - 1503s 25s/step - loss: 1.1079 - accuracy: 0.6105 - val_loss: 1.0997 - val_accuracy: 0.5203
Epoch 7/30
60/60 [==============================] - 1456s 24s/step - loss: 1.0604 - accuracy: 0.6194 - val_loss: 1.0640 - val_accuracy: 0.4593

In [ ]:
# 체크포인트 불러오기
epochs = 15

model = modeling(input_shape = image_size + (3,), num_classes=num_classes)
model.load_weights(work_dir + 'model/v10/save_at_14.h5')

model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"],)
result = model.fit(train_ds,epochs=epochs,callbacks=[early_stopping, model_checkpoint],validation_data=val_ds,)

model.save(model_dir + model_name)

Epoch 1/15
60/60 [==============================] - 1507s 25s/step - loss: 0.6979 - accuracy: 0.7016 - val_loss: 0.7633 - val_accuracy: 0.5407
Epoch 2/15
60/60 [==============================] - 82s 1s/step - loss: 0.6480 - accuracy: 0.6942 - val_loss: 0.7360 - val_accuracy: 0.5569
Epoch 3/15
60/60 [==============================] - 78s 1s/step - loss: 0.6426 - accuracy: 0.6979 - val_loss: 0.7394 - val_accuracy: 0.5366
Epoch 4/15
60/60 [==============================] - 79s 1s/step - loss: 0.6271 - accuracy: 0.6948 - val_loss: 0.7241 - val_accuracy: 0.5650
Epoch 5/15
60/60 [==============================] - 78s 1s/step - loss: 0.6184 - accuracy: 0.7031 - val_loss: 0.7516 - val_accuracy: 0.5569
Epoch 6/15
60/60 [==============================] - 76s 1s/step - loss: 0.6203 - accuracy: 0.7152 - val_loss: 0.7672 - val_accuracy: 0.5325
Epoch 7/15
60/60 [==============================] - 79s 1s/step - loss: 0.5998 - accuracy: 0.7073 - val_loss: 0.7658 - val_accuracy: 0.5935
Epoch 8/15
60/60 

In [ ]:
# 학습된 모델 불러와서 이어서 학습
model = load_model(model_dir + model_name)

model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"],)
result = model.fit(train_ds,epochs=epochs,callbacks=[early_stopping, model_checkpoint],validation_data=val_ds,)

model.save(model_dir + model_name)